# MDTraj and OpenMM
This example builds on the MDTraj "Simulation with OpenMM" example (http://mdtraj.org/latest/examples/openmm.html) to set up a very simple simulation of a tryptophan zipper with the AMBER force fields in implicit solvent, run a few steps of dynamics, and visualize the result. 

Note you will need to conda install several things before you can use this:
* `conda install -c omnia openmm`
* `conda install -c omnia mdtraj` 
* `conda install -c bioconda nglview`

It is possible you will also need to issue the command `jupyter-nbextension enable nglview --py --sys-prefix`

In [4]:
import os
import mdtraj
import mdtraj.reporters
from simtk import unit #Units is an extremely useful module that allows numbers to carry around assigned units
import simtk.openmm as mm
from simtk.openmm import app

In [5]:
#Load PDB NMR structure of mini-protein tryptophan zipper, AKA TRP-ZIP.
pdb = mdtraj.load_pdb('http://www.rcsb.org/pdb/files/1LE1.pdb')

#Convert PDB to openmm format
topology = pdb.topology.to_openmm()
#Assign AMBER force field parameters
forcefield = app.ForceField('amber99sbildn.xml', 'amber99_obc.xml')

#Set up an OpenMM System with non-periodic boundary conditions (simplest setup, not generally advisable)
system = forcefield.createSystem(topology, nonbondedMethod=app.CutoffNonPeriodic)

#We'll use Langevin dynamics to simulate/as the thermostat; 1.0/ps friction and 2.0 femtosecond timestep. Temperature 330K
integrator = mm.LangevinIntegrator(330*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
#Initialize simulation
simulation = app.Simulation(topology, system, integrator)
#Set starting coordinates
simulation.context.setPositions(pdb.xyz[0])
#Assign random starting velocities from correct distribution
simulation.context.setVelocitiesToTemperature(330*unit.kelvin)

#Set up storage of trajectories, storing snapshots every 10 frames to the HDF5 format
reporter = mdtraj.reporters.HDF5Reporter('test.h5', 10)
simulation.reporters.append(reporter)

#Run 100 steps of dynamics (you can increase this number if you want)
simulation.step(100)

#Close "reporter" file storing trajectory info
reporter.close()
print("Done")

//anaconda/lib/python3.5/site-packages/mdtraj/formats/pdb/pdbfile.py:196: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


Done


## Now use the NGLView widget/tool to embed visualization

In [6]:
from __future__ import print_function
import nglview #Embedded viewer

#Load test trajectory
traj = mdtraj.load('test.h5')

#Print info on what's in our trajectory
print(traj)

#Load MDTraj trajectory into view (it's also possible to visualize PDB's
# and a variety of other formats)
view = nglview.show_mdtraj(traj)

#Add space filling representation (optional)
view.add_representation('spacefill', selection='all' )

#Other things you might want to try
view.clear_representations()
view.add_licorice('not hydrogen') #Try 'not hydrogen' or 'all' for example
view.add_ribbon()



#This view command needs to be LAST OF ALL or you will get a blank viewer
view


#VIEWER USAGE:
# - Use your typical zoom command/gesture (i.e. pinch) to zoom in and out 
# - Click and drag to reorient
# - Click on specific atoms/residues to find out details of what they are (and how they could be selected)

<mdtraj.Trajectory with 10 frames, 220 atoms, 13 residues, without unitcells>


## Other viewer usage notes

The nglview viewer seems to be relatively full featured (https://github.com/arose/nglview), but is not that well documented yet. Some usage notes which might or might not be useful include:

In [ ]:
#It is possible to add additional trajectories to your view, such as using:
#view.add_trajectory(nglview.MDTrajTrajectory(traj2))

#Other "trajectories" don't have to be of the same type; for example, you could load a `pytraj` 
#trajectory from AMBER's pytraj module, or you can even load a molecular structure from the RDKit
#toolkit (an open source Chem toolkit somewhat like OpenEye's toolkits), such as via:
#view.add_trajectory(nglview.RdkitStructure(its_object))

#See additional examples at https://github.com/arose/nglview/blob/master/examples/representations_and_selection_language.ipynb